In [21]:
import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

# Data Preprocessing 

In [22]:
single_pdf_path = "/home/kamal/Desktop/HelpMateAI/app/data/Principal-Sample-Life-Insurance-Policy.pdf"

In [23]:
cd ..

/home/kamal/Desktop


/home/kamal/Desktop/HelpMateAI/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [24]:
from app.helpmateAI.services.dataextraction import extract_text_from_pdf

In [25]:
df = extract_text_from_pdf(single_pdf_path=single_pdf_path)

In [26]:
import os
documents_name = os.listdir("/home/kamal/Desktop/HelpMateAI/app/data")
doc_name = documents_name[0]
doc_name

'Principal-Sample-Life-Insurance-Policy.pdf'

In [27]:
# document name column created
df['document_name'] = doc_name

In [28]:
## function to count word of the sentences
def text_count(text):
    words = text.strip().split()
    words_count = len(words)
    return words_count

In [29]:
## total pages numbers
df["total_words"] = df["page_context"].apply(lambda x: text_count(x))

In [30]:
## filtered data which has more than total 10 words
df = df[df["total_words"] >= 10]
df.shape

(60, 4)

In [31]:
# Store the metadata for each page in a separate column
file_name = df["document_name"][0][:-4]
df["metadata"] =df.apply(lambda x: {"Policy_Name":file_name, "Page_No.":x["page_number"]}, axis=1)

## ChromaDB

In [37]:
from app.helpmateAI.services.chromadb import create_and_store_chromadb_data

chromadb_collection = create_and_store_chromadb_data(chromadb_name="HelpMateAI_collection",
                                                     db_path="/home/kamal/Desktop/HelpMateAI/app/db/chromadb/",
                                                     dataframe=df
                                                     )

In [38]:
chromadb_collection.name

'HelpMateAI_collection'

In [39]:
chromadb_collection.count()

60

# cachedb

In [40]:
from app.helpmateAI.services.cache import initialize_cache_chromadb

cache_collection = initialize_cache_chromadb(chromadb_name="HelpMate_cache_collection",
                                             db_path="/home/kamal/Desktop/HelpMateAI/app/db/chromadb_cache/")

In [41]:
cache_collection.name

'HelpMate_cache_collection'

# sementic search with cachedb

In [42]:
# Query the collection against the user query and return the top 20 results

# query = "what is death policy"
# query = "What purpose do the Certificates serve regarding insurance?"
query = "When can The Principal inspect the Policyholder records related to the Group Policy?"
cache_results = cache_collection.query(
    query_texts=[query],
    n_results=1
)

# cache initializtion

In [43]:
from app.helpmateAI.services.cache import generate_cache_initialized_dataframe

result_df = generate_cache_initialized_dataframe(query=query,
                                     threshold=0.2,
                                     chromadb_obj=chromadb_collection,
                                     cache_query_results=cache_results,
                                     cache_collector_obj=cache_collection,
                                     )

Not found in cache. Found in main collections


In [44]:
result_df

,Metadatas,Documents,Distances,IDs
0,"{'Page_No.': 'Page 16', 'Policy_Name': 'Princi...",PART II - POLICY ADMINISTRATION Section A - Co...,0.842683,ids-13
1,"{'Page_No.': 'Page 19', 'Policy_Name': 'Princi...",T he Principal has complete discretion to cons...,0.911736,ids-16
2,"{'Page_No.': 'Page 24', 'Policy_Name': 'Princi...",T he Principal may terminate the Policyholder'...,0.943317,ids-21
3,"{'Page_No.': 'Page 5', 'Policy_Name': 'Princip...",PRINCIPAL LIFE INSURANCE COMPANY (called The P...,0.970175,ids-2
4,"{'Page_No.': 'Page 18', 'Policy_Name': 'Princi...",c . a copy of the form which contains the stat...,1.075677,ids-15
5,"{'Page_No.': 'Page 6', 'Policy_Name': 'Princip...",TABLE OF CONTENTS PART I - DEFINITIONS PART II...,1.098102,ids-3
6,"{'Page_No.': 'Page 3', 'Policy_Name': 'Princip...",POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,1.098731,ids-1
7,"{'Page_No.': 'Page 34', 'Policy_Name': 'Princi...",provided The Principal has been notified of th...,1.131059,ids-31
8,"{'Page_No.': 'Page 22', 'Policy_Name': 'Princi...",The number of Members insured for Dependent Li...,1.183612,ids-19
9,"{'Page_No.': 'Page 23', 'Policy_Name': 'Princi...",Section C - Policy Termination Article 1 - Fai...,1.185161,ids-20


## Coress reranker

In [45]:
from app.helpmateAI.services.reranker import sentence_cross_encoder

reranker_score = sentence_cross_encoder(result_dataframe=result_df,
                                        query=query)

reranker_score

/home/kamal/Desktop/HelpMateAI/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-30 17:12:04.390956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735558924.717752   14712 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735558924.938019   14712 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


array([ 1.171242 ,  1.7042   ,  2.420164 ,  2.4954324,  3.2597113,
       -7.2790937,  0.7326738, -1.1463788, -2.80421  ,  1.5917959],
      dtype=float32)

## Top 3 generator

In [46]:
from app.helpmateAI.services.retrieval import top_3_relevant_documents

top_3_docs, reranker_docs = top_3_relevant_documents(cross_rerank_scores=reranker_score,result_df=result_df)

In [47]:
top_3_docs

,Documents,Metadatas
4,c . a copy of the form which contains the stat...,"{'Page_No.': 'Page 18', 'Policy_Name': 'Princi..."
3,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,"{'Page_No.': 'Page 5', 'Policy_Name': 'Princip..."
2,T he Principal may terminate the Policyholder'...,"{'Page_No.': 'Page 24', 'Policy_Name': 'Princi..."


In [48]:
reranker_docs

,Metadatas,Documents,Distances,IDs,Reranked_scores
4,"{'Page_No.': 'Page 18', 'Policy_Name': 'Princi...",c . a copy of the form which contains the stat...,1.075677,ids-15,3.259711
3,"{'Page_No.': 'Page 5', 'Policy_Name': 'Princip...",PRINCIPAL LIFE INSURANCE COMPANY (called The P...,0.970175,ids-2,2.495432
2,"{'Page_No.': 'Page 24', 'Policy_Name': 'Princi...",T he Principal may terminate the Policyholder'...,0.943317,ids-21,2.420164
1,"{'Page_No.': 'Page 19', 'Policy_Name': 'Princi...",T he Principal has complete discretion to cons...,0.911736,ids-16,1.704200
9,"{'Page_No.': 'Page 23', 'Policy_Name': 'Princi...",Section C - Policy Termination Article 1 - Fai...,1.185161,ids-20,1.591796
0,"{'Page_No.': 'Page 16', 'Policy_Name': 'Princi...",PART II - POLICY ADMINISTRATION Section A - Co...,0.842683,ids-13,1.171242
6,"{'Page_No.': 'Page 3', 'Policy_Name': 'Princip...",POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,1.098731,ids-1,0.732674
7,"{'Page_No.': 'Page 34', 'Policy_Name': 'Princi...",provided The Principal has been notified of th...,1.131059,ids-31,-1.146379
8,"{'Page_No.': 'Page 22', 'Policy_Name': 'Princi...",The number of Members insured for Dependent Li...,1.183612,ids-19,-2.804210
5,"{'Page_No.': 'Page 6', 'Policy_Name': 'Princip...",TABLE OF CONTENTS PART I - DEFINITIONS PART II...,1.098102,ids-3,-7.279094


## response

In [51]:
from app.helpmateAI.services.generate import response_generator
# query = "what is death policy"
# query = "What purpose do the Certificates serve regarding insurance?"
query = "When can The Principal inspect the Policyholder records related to the Group Policy?"
response = response_generator(query=query, top_3_RAG=top_3_docs, api_key=api_key)

In [52]:
response

['Based on the provided search results, the Principal may inspect the Policyholder records related to the Group Policy as mentioned in the document.',
 '',
 'According to the document, "The Principal may terminate the Policyholder\'s rights under this Policy if, in its reasonable opinion, such action is necessary to avoid any liability or cost which may be incurred by the Principal." (Source: Page 24, Policy Name: "Principal Executive Life Assurance Corporation").',
 '',
 'The relevant policy name and page number are:',
 '',
 '* Policy Name: "Principal Executive Life Assurance Corporation"',
 '* Page Number: 24',
 '',
 'This information answers the query directly. I hope this is accurate and helpful.']

## result

In [54]:
def get_top_3_docs(query: str):

    cache_results = cache_collection.query(
        query_texts=[query],
        n_results=1
    )

    result_df = generate_cache_initialized_dataframe(query=query,
                                        threshold=0.2,
                                        chromadb_obj=chromadb_collection,
                                        cache_query_results=cache_results,
                                        cache_collector_obj=cache_collection,
                                        )

    reranker_score = sentence_cross_encoder(result_dataframe=result_df,
                                            query=query)

    top_3_docs, _ = top_3_relevant_documents(cross_rerank_scores=reranker_score,result_df=result_df)

    return top_3_docs

In [60]:
query = "What information must the Policyholder provide to The Principal upon request?"

top_3_docs = get_top_3_docs(query=query)

Not found in cache. Found in main collections


In [66]:
top_3_docs

,Documents,Metadatas
1,c . a copy of the form which contains the stat...,"{'Page_No.': 'Page 18', 'Policy_Name': 'Princi..."
4,T he Principal may terminate the Policyholder'...,"{'Page_No.': 'Page 24', 'Policy_Name': 'Princi..."
0,PART II - POLICY ADMINISTRATION Section A - Co...,"{'Page_No.': 'Page 16', 'Policy_Name': 'Princi..."


In [74]:
query = "What purpose do the Certificates serve regarding insurance?"
response = response_generator(query=query, top_3_RAG=top_3_docs, api_key=api_key)

In [83]:
response

["The certificates in an insurance policy serve as a proof of policy ownership and coverage. They typically provide essential details about the policy, such as the policyholder's name, the policy's coverage limits, and the effective dates.",
 '',
 'According to the documents provided, it is evident that certificates are issued to provide a summary of the policy terms and conditions. For instance, Certificate No. 1, found on Page 18 of the "Principal" policy, contains a copy of the form that specifies the statistics and details of the policy.',
 '',
 'Table 1: Key information from Certificate No. 1',
 '',
 '| Policy Details | Information |',
 '| --- | --- |',
 '| Policyholder Name | [Name] |',
 '| Coverage Limits | [Amount] |',
 '| Effective Dates | [Start Date] - [End Date] |',
 '',
 'Policy Names and Page Numbers:',
 '',
 '* "Principal" policy: Page 18',
 '* [Additional policies with certificates mentioned but not found in this snippet]',
 '',
 'Please refer to these sections of the p

In [ ]:
# query = "what is death policy"
# query = "What purpose do the Certificates serve regarding insurance?"
# query = "What information must the Policyholder provide to The Principal upon request?"